<a href="https://colab.research.google.com/github/aandrks/process_coordination/blob/main/processing_coordinations_program.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Coordinations Processing
import re
import json
import pandas as pd
from datetime import datetime, timedelta
from collections import defaultdict
from pathlib import Path
import os
import traceback
import unicodedata
!pip install rapidfuzz
from rapidfuzz import fuzz, process
from google.colab import files
!pip install xlsxwriter

# Configuration - use relative paths for Colab
EMPLOYEE_DB_FILE = 'employee_database.json'
public_domains = {'mail', 'yandex', 'gmail', 'yahoo', 'hotmail', 'outlook'}
no_match_array = []
holidays = ['01-01', '02-01', '03-01', '04-01', '05-01', '06-01', '07-01', '23-02', '08-03', '01-05', '09-05', '12-06', '03-11', '04-11'] # ADD HOLIDAYS
working_holidays = ['01-11'] # ADD EXTRA WORKING DAYS


def load_employee_db():
    """Load employee database from file"""
    try:
        if Path(EMPLOYEE_DB_FILE).exists():
            with open(EMPLOYEE_DB_FILE, 'r', encoding='utf-8') as f:
                db = json.load(f)
                # Convert companies list back to set
                db['companies'] = set(db['companies'])
                print(
                    f"✓ Loaded employee database with {len(db['employees'])} employees and {len(db['companies'])} companies")
                return db
    except Exception as e:
        print(f"⚠️ Error loading employee database: {e}")
    return {'employees': [], 'companies': set()}


def save_employee_db(db):
    """Save employee database to file"""
    try:
        # Convert sets to lists for JSON serialization
        db_to_save = {
            'employees': db['employees'],
            'companies': list(db['companies'])
        }
        with open(EMPLOYEE_DB_FILE, 'w', encoding='utf-8') as f:
            json.dump(db_to_save, f, ensure_ascii=False, indent=2)
        print(f"✓ Saved employee database with {len(db['employees'])} employees and {len(db['companies'])} companies")
    except Exception as e:
        print(f"⚠️ Error saving employee database: {e}")


def upload_file(file_type="any"):
    """Upload a file in Google Colab"""
    print(f"Please upload {file_type} file:")
    uploaded = files.upload()
    if uploaded:
        filename = list(uploaded.keys())[0]
        print(f"✓ Uploaded: {filename}")
        return filename
    else:
        print("No file uploaded")
        return None


def normalize_text(text):
    """Normalize text by removing accents, special characters, and converting to lowercase"""
    if not isinstance(text, str):
        return ""

    # Remove accents and diacritics
    text = unicodedata.normalize('NFKD', text)
    text = ''.join([c for c in text if not unicodedata.combining(c)])

    # Remove all non-alphanumeric characters except spaces and dots (for initials)
    text = re.sub(r'[^\w\s.]', '', text)

    # Convert to lowercase
    return text.lower().strip()


def is_initial(part):
    """Check if a name part is an initial"""
    return len(part) <= 2 or (len(part) == 2 and part.endswith('.'))


def extract_name_components(name):
    """Extract surname and given names from a full name with proper handling of initials"""
    if not isinstance(name, str):
        return "", ""

    # Remove any non-alphabetic characters except spaces and dots
    clean_name = re.sub(r'[^а-яА-ЯёЁa-zA-Z\s.]', '', name).strip()

    # Handle comma-separated names (Last, First format)
    if ',' in clean_name:
        parts = [p.strip() for p in clean_name.split(',')]
        if len(parts) >= 2:
            return parts[0], parts[1]  # surname, given names

    # Split into words and remove empty parts
    parts = [p for p in re.split(r'\s+', clean_name) if p]

    if not parts:
        return "", ""
    if len(parts) == 1:
        return parts[0], ""

    # Check if the last part is an initial
    if is_initial(parts[-1]):
        # Format: "Surname Initial" or "Surname I."
        return parts[0], parts[-1]

    # Check if the first part is an initial
    if is_initial(parts[0]):
        # Format: "Initial Surname" or "I. Surname"
        return parts[-1], parts[0]

    # If no initials, assume last word is surname
    return parts[-1], " ".join(parts[:-1])


def parse_company_person_data(file_content, db):
    """Process company and employee data with enhanced name handling and team support"""
    company_person_map = defaultdict(list)
    new_employees = []
    seen_emails = {e['email'] for e in db['employees']}
    processing_log = []
    team_id_counter = 1  # Counter for generating unique team IDs

    print("\nProcessing company and employee data...")
    print("=" * 50)
    processing_log.append("Processing company and employee data...")
    processing_log.append("=" * 50)

    for line_num, line in enumerate(file_content.split('\n'), 1):
        if not line.strip():
            continue

        original_line = line.strip()
        lines = file_content.split('\n')

    for line_num in range(len(lines)):
        line = lines[line_num].strip()
        if not line:
            continue

        # NEW: Check if this line ends with / and combine with next line
        if line.endswith('/') and line_num + 1 < len(lines):
            next_line = lines[line_num + 1].strip()
            if next_line:
                line = line.rstrip('/') + ' ' + next_line
                # Skip the next line since we've combined it
                lines[line_num + 1] = ""
        original_line = line
        processing_log.append(f"\nLine {line_num + 1}: {original_line}")

        # Find all email blocks in parentheses
        for block in re.findall(r'(?:\(| - )([^()]+?\s+[^\s@]+@[^\s/@]+(?:\s*/\s*[^()]+?\s+[^\s@]+@[^\s/@]+)*)', line):
            if '@' not in block:
                continue

            processing_log.append(f"  Found email block: {block}")

            # Extract team members (people separated by '/')
            team_members = [p.strip() for p in block.split('/')]
            team_id = f"team_{team_id_counter}"
            team_id_counter += 1

            # Process each person in the team
            team_company = None
            team_emails = []

            for person in team_members:
                match = re.search(r'([^@]+)\s+([^\s@]+@[^\s@]+)', person)
                if not match:
                    processing_log.append(f"    Skipping malformed person entry: {person}")
                    continue

                name, email = match.group(1).strip(), match.group(2).strip()
                email = re.sub(r'[),.;]+$', '', email)
                email = email.strip()
                processing_log.append(f"    Processing: {name} <{email}>")

                if email in seen_emails:
                    processing_log.append(f"    Skipping duplicate email: {email}")
                    continue
                seen_emails.add(email)
                team_emails.append(email)

                domain = email.split('@')[-1].split('.')[0]
                surname, given_names = extract_name_components(name)
                normalized_name = normalize_text(name)

                processing_log.append(f"    Extracted: surname='{surname}', given_names='{given_names}'")

                if domain in public_domains:
                    processing_log.append(f"    Public domain detected: {domain}")
                    print(f"\nFound public domain email:")
                    print(f"Original line: {original_line}")
                    print(f"Employee: {name} <{email}>")

                    while True:
                        company = input("Assign company (leave blank to skip, 'q' to quit): ").strip()

                        if company.lower() == 'q':
                            # Save before quitting
                            db['employees'].extend(new_employees)
                            db['companies'].update(e['company'] for e in new_employees)
                            save_employee_db(db)
                            processing_log.append("User quit during manual assignment")
                            return db, company_person_map, processing_log

                        if not company:
                            processing_log.append(f"    Skipped {email}")
                            print(f"Skipped {email}")
                            break

                        if len(company) < 2:
                            print("Company name too short, try again")
                            continue

                        # If this is the first team member with a manually assigned company,
                        # use this company for the whole team
                        if team_company is None:
                            team_company = company
                            db['companies'].add(company)

                        new_employees.append({
                            'name': name,
                            'email': email,
                            'normalized_name': normalized_name,
                            'surname': surname,
                            'given_names': given_names,
                            'company': company,
                            'source': 'manual',
                            'team_id': team_id,
                            'team_emails': team_emails
                        })
                        company_person_map[company].append({
                            'name': name,
                            'email': email,
                            'normalized_name': normalized_name,
                            'surname': surname,
                            'given_names': given_names,
                            'team_id': team_id,
                            'team_emails': team_emails
                        })
                        processing_log.append(f"    Assigned to company: {company}")
                        print(f"✓ Assigned: {company}")
                        break
                else:
                    # If this is the first team member with a company domain,
                    # use this company for the whole team
                    if team_company is None:
                        team_company = domain
                        db['companies'].add(domain)

                    new_employees.append({
                        'name': name,
                        'email': email,
                        'normalized_name': normalized_name,
                        'surname': surname,
                        'given_names': given_names,
                        'company': domain,
                        'source': 'auto',
                        'team_id': team_id,
                        'team_emails': team_emails
                    })
                    company_person_map[domain].append({
                        'name': name,
                        'email': email,
                        'normalized_name': normalized_name,
                        'surname': surname,
                        'given_names': given_names,
                        'team_id': team_id,
                        'team_emails': team_emails
                    })
                    processing_log.append(f"    Auto-assigned to domain: {domain}")
                    print(f"✓ Auto-assigned: {name} <{email}> → {domain}")

    # Update database with new employees
    db['employees'].extend(new_employees) # extending
    # db['employees'] = new_employees # exchanging
    # change type of loading data update
    save_employee_db(db)
    processing_log.append(f"\nAdded {len(new_employees)} new employees to database")

    return db, company_person_map, processing_log


def find_best_match(target_name, candidates, debug_info=None):
    """Find the best match using fuzzy matching on full name"""
    if debug_info is None:
        debug_info = []

    debug_info.append(f"    Finding best match for: {target_name}")
    debug_info.append(f"    Candidates: {[c['name'] for c in candidates]}")

    # Extract components from target name for potential exact surname matching
    target_surname, target_given = extract_name_components(target_name)
    debug_info.append(f"    Target surname: {target_surname}, given: {target_given}")

    # If target has an initial, expand it to possible full names
    target_possible_givens = []
    target_possible_givens = [target_given]

    # First try: exact surname match + given name similarity
    for candidate in candidates:
        # Match surname exactly
        if normalize_text(target_surname) != normalize_text(candidate['surname']):
            continue

        # If target has an initial, check if candidate's given name matches any expansion
        candidate_given_norm = normalize_text(candidate['given_names'])

        for possible_given in target_possible_givens:
            possible_given_norm = normalize_text(possible_given)

            # Check if candidate's given name starts with any of the possible expansions
            if candidate_given_norm.startswith(possible_given_norm):
                debug_info.append(f"    Initial-based match: {candidate['name']} "
                                  f"(target: {target_given}, candidate: {candidate['given_names']})")
                return candidate

            # Check if any of the possible expansions starts with candidate's given name
            if possible_given_norm.startswith(candidate_given_norm):
                debug_info.append(f"    Initial-based match: {candidate['name']} "
                                  f"(target: {target_given}, candidate: {candidate['given_names']})")
                return candidate

    # Second try: fuzzy matching on full name (using the complete name from JSON)
    best_score = 0
    best_match = None
    for candidate in candidates:
        # Use the full normalized name from the database for matching
        score = fuzz.token_set_ratio(normalize_text(target_name), candidate['normalized_name'])
        debug_info.append(f"    Fuzzy score for {candidate['name']}: {score}")
        if score > 65 and score > best_score:  # Increased threshold for better accuracy
            best_score = score
            best_match = candidate

    if best_match:
        debug_info.append(f"    Fuzzy match selected: {best_match['name']} (score: {best_score})")
    else:
        debug_info.append("    No suitable match found")

    return best_match


def add_working_days(start_date, working_days):
    """Add working days excluding weekends (Saturday and Sunday)"""
    if working_days <= 0:
        return start_date

    current_date = start_date
    days_added = 0
    monthday = current_date.strftime("%d-%m")

    while days_added < working_days:
        current_date += timedelta(days=1)
        # Check if it's a weekday (Monday=0, Sunday=6)
        if (current_date.weekday() < 5) and (monthday not in holidays):  # 0-4 are Monday-Friday
            days_added += 1
        elif(monthday in working_holidays):
            days_added += 1
        else:
            # It's a weekend, skip counting but continue to next day
            continue

    return current_date

SPEC_CONFIG_FILE = 'spec_config.json'

def load_spec_config():
    """Load specification configuration from file"""
    try:
        if Path(SPEC_CONFIG_FILE).exists():
            with open(SPEC_CONFIG_FILE, 'r', encoding='utf-8') as f:
                spec_config = json.load(f)
                print(f"✓ Loaded specification configuration")
                return spec_config
    except Exception as e:
        print(f"⚠️ Error loading specification configuration: {e}")

    # Return default structure if file doesn't exist
    return {
        "2": {
            "раздела КР": 2
        },
        "3": {

        },
        "4": {
        }
    }

def get_working_days(step_text, workflow_text):
    """Calculate working days based on stage and specification keywords"""
    # First check for step number
    stage_match = re.search(r'Шаг (\d+)', step_text)
    if "Утверждение" in step_text:
      return (2, 4, "Stage 4")
    elif not stage_match:
        return (0, 0, "No stage number found")

    step_number = int(stage_match.group(1))
    stage_number = step_number + 1  # Шаг 1 → stage 2, Шаг 2 → stage 3


    # Change keywords
    spec_config = load_spec_config()

    # Default days for each stage if no keywords found
    default_days = {
        2: 3,
        3: 5,
        4: 2
    }

    # Check if this stage has configured keywords
    if stage_number not in spec_config:
        days = default_days.get(stage_number, 0)
        return (days, stage_number, f"Stage {stage_number}: not configured, using default {days} days")

    # Get keywords for this specific stage
    stage_keywords = spec_config[stage_number]
    workflow_lower = workflow_text.lower()

    # Check each keyword for this stage
    for keyword, days in stage_keywords.items():
        if keyword.lower() in workflow_lower:
            return (days, stage_number, f"Stage {stage_number}: keyword '{keyword}' → {days} days")

    # No keywords found for this stage, use default value
    days = default_days.get(stage_number, 0)
    return (days, stage_number, f"Stage {stage_number}: no keywords found, using default {days} days")


def extract_start_date_from_lifecycle(lifecycle_text, current_step_number):
    """Extract start date from lifecycle text based on previous stage completion"""
    if not lifecycle_text or pd.isna(lifecycle_text):
        return None


    current_stage = current_step_number + 1
    target_step = current_stage - 2  # Stage 3 → Шаг 1, Stage 2 → Шаг 0

    if target_step < 0:
        return None

    # Pattern to find step entries with dates
    step_pattern = rf'Шаг {target_step}.*?(\d{{2}}\.\d{{2}}\.\d{{2}} \d{{2}}:\d{{2}})'
    matches = re.findall(step_pattern, lifecycle_text, re.IGNORECASE | re.DOTALL)

    if matches:
        # Take the last date (most recent completion)
        last_date_str = matches[-1]
        try:
            # Convert from "DD.MM.YY HH:MM" to datetime
            return datetime.strptime(last_date_str, '%d.%m.%y %H:%M')
        except ValueError:
            return None

    return None


def get_date():
    a1 = input("\n\n\nType date in format dd.mm.YYYY \nor pull Enter if today is ok: ")
    if(len(a1) < 2):
        return datetime.today().date()
    else:
      try:
        return datetime.strptime(a1, "%d.%m.%Y").date()
      except ValueError:
        print("Date is not correct, today has been assigned as today")
        return datetime.today().date()


def is_team_checked(approver_name, all_people, checked_approvers, matching_log):
    """Check if any team member of the given approver is already checked"""
    # First find the best match for the approver
    best_match = find_best_match(approver_name, all_people, matching_log)
    if not best_match:
        matching_log.append(f"    No match found for team check: {approver_name}")
        return False

    # Check if this person has a team
    team_id = best_match.get('team_id')
    team_emails = best_match.get('team_emails', [])

    if not team_id or len(team_emails) <= 1:
        matching_log.append(f"    No team found for: {approver_name}")
        return False

    matching_log.append(f"    Checking team {team_id} with {len(team_emails)} members")

    # Find all team members in the database
    team_members = []
    for person in all_people:
        if person.get('team_id') == team_id:
            team_members.append(person)
            matching_log.append(f"      Team member: {person['name']} <{person['email']}>")

    # Check if any team member is in the checked approvers list
    for team_member in team_members:
        # Use fuzzy matching to see if team member name is in checked approvers
        for checked_name in checked_approvers:
            if find_best_match(checked_name, [team_member], matching_log):
                matching_log.append(f"    ✓ Team member {team_member['name']} is already checked")
                return True

    matching_log.append(f"    ✗ No team members found in checked list")
    return False


def process_coordinations(df, company_person_map):
    """Process coordinations with disambiguation for same surnames and team handling"""
    overdue_counts = defaultdict(int)
    overdue_emails = []
    overdue_coordination_ids = []
    coordination_details = []
    result_df = []
    # df1 = pd.DataFrame(columns = [])



    # Change today's date

    # = datetime.today().date()

    # = datetime.strptime("11.09.2025", "%d.%m.%Y").date()

    today = get_date()

    # datetime.strptime("26.09.2025", "%d.%m.%Y").date()




    debug_info = []
    ambiguous_matches = []
    matching_log = []

    print(f"\nProcessing coordinations (as of {today})... {datetime.today()}")
    matching_log.append(f"Processing coordinations (as of {today})...")
    matching_log.append("=" * 50)

    # Prepare a flat list of all people for matching
    all_people = []
    for company, persons in company_person_map.items():
        for person in persons:
            person['company'] = company
            all_people.append(person)

    # Get coordination ID column name (first column)
    id_column = df.columns[0] if len(df.columns) > 0 else 'id'
    print(f"Using '{id_column}' as coordination ID")
    matching_log.append(f"Using '{id_column}' as coordination ID")

    for idx, row in df.iterrows():
        if not all(col in row for col in ['Не проверили на текущем шаге', 'Шаг', 'Рабочий процесс']):
            matching_log.append(f"Row {idx}: Missing required columns, skipping")
            continue

        coord_id = row.get(id_column, 'N/A')
        step_text = str(row['Шаг'])
        workflow_text = str(row['Рабочий процесс'])

        matching_log.append(f"\nProcessing coordination ID: {coord_id}")
        matching_log.append(f"Step: {step_text}, Workflow: {workflow_text}")

        # Get working days with explanation
        working_days, stage_number, days_explanation = get_working_days(step_text, workflow_text)
        matching_log.append(f"Working days calculation: {days_explanation}")

        try:
            # First try to get start date from lifecycle (previous stage completion)
            start_date = None
            if 'Жизненный цикл' in row and row['Жизненный цикл']:
                lifecycle_text = str(row['Жизненный цикл'])
                step_match = re.search(r'Шаг (\d+)', step_text)

                if("Утверждение" in step_text):
                    current_step_number = 3
                    start_date = extract_start_date_from_lifecycle(lifecycle_text, current_step_number)
                elif step_match:
                    current_step_number = int(step_match.group(1))

                    start_date = extract_start_date_from_lifecycle(lifecycle_text, current_step_number)
                    if start_date:
                        matching_log.append(f"  Found start date from lifecycle: {start_date}")
                    else:
                        matching_log.append("  No valid start date found in lifecycle")

            # If no start date from lifecycle, use creation date
            if start_date is None:
                start_date_str = str(row['Дата и время создания согласования'])
                start_date = datetime.strptime(start_date_str, '%Y-%m-%d %H:%M:%S')
                matching_log.append(f"  Using creation date as start date: {start_date}")

            deadline = add_working_days(start_date, working_days)
            matching_log.append(f"Start date: {start_date.date()}, Deadline: {deadline.date()}")

            # Skip if not overdue
            if deadline.date() >= today:
                debug_info.append({
                    'id': coord_id,
                    'status': 'Not overdue',
                    'start_date': start_date.date(),
                    'deadline': deadline.date(),
                    'working_days': working_days,
                    'explanation': days_explanation,
                    'today': today
                })
                matching_log.append("Coordination is not overdue, skipping")
                continue

        except Exception as e:
            debug_info.append({
                'id': coord_id,
                'status': f'Date error: {str(e)}',
                'start_date_str': start_date_str if 'start_date_str' in locals() else 'N/A',
                'working_days': working_days,
                'explanation': days_explanation
            })
            matching_log.append(f"Date parsing error: {str(e)}")
            continue

        # Get NOT checked approvers from "Не проверили на текущем шаге" column
        not_checked_text = str(row['Не проверили на текущем шаге'])
        not_checked_approvers = [name.strip() for name in not_checked_text.split(',') if name.strip()]

        matching_log.append(f"Not checked approvers: {not_checked_approvers}")

        # Get checked approvers from "Проверили на текущем шаге" column
        checked_text = str(row['Проверили на текущем шаге'])
        checked_approvers = [name.strip() for name in checked_text.split(',') if name.strip()]

        matching_log.append(f"Checked approvers: {checked_approvers}")

        # Find email matches for not checked approvers
        coord_emails = []
        coord_companies = set()
        checked_members = []
        for approver_name in not_checked_approvers:
            # Checking if anyone from the appover's team has already finished
            if is_team_checked(approver_name, all_people, checked_approvers, matching_log):
                matching_log.append(f"    Skipping {approver_name} - team member already checked")
                checked_members.append(f"{approver_name} checked!")
                continue


            # Use fuzzy matching on the full name against all people
            best_match = find_best_match(approver_name, all_people, matching_log)
            if best_match:
                coord_emails.append(best_match['email'])
                coord_companies.add(best_match['company'])
                matching_log.append(f"    Matched: {approver_name} → {best_match['name']} <{best_match['email']}>")
            else:
                matching_log.append(f"    No match found for: {approver_name}")
                no_match_array.append(approver_name)

        # Update overdue counts and emails
        for company in coord_companies:
            overdue_counts[company] += 1
        overdue_emails.extend(coord_emails)
        overdue_coordination_ids.append(coord_id)

        # Store coordination details
        coordination_details.append({
            'id': coord_id,
            'company': ', '.join(coord_companies),
            'start_date': start_date.date(),
            'deadline': deadline.date(),
            'working_days': working_days,
            'not_checked_count': len(not_checked_approvers),
            'explanation': days_explanation,
            'emails': coord_emails
        })

        debug_info.append({
            'id': coord_id,
            'status': 'Overdue',
            'start_date': start_date.date(),
            'deadline': deadline.date(),
            'working_days': working_days,
            'explanation': days_explanation,
            'companies': list(coord_companies),
            'checked_members' : checked_members
        })
        result_df.append({'id' : row['id'],
                          'Согласование' : row['Согласование'],
                          'Шаг' : row['Шаг'],
                          'Срок текущего шага' : row['Срок текущего шага'],
                          'Дата и время создания согласования' : row['Дата и время создания согласования'],
                          'Истечение срока согласования' : row['Истечение срока согласования'],
                          'Инициатор' : row['Инициатор'],
                          'Согласующие' : row['Согласующие'],
                          'Проверили на текущем шаге' : row['Проверили на текущем шаге'],
                          'Не проверили на текущем шаге' : row['Не проверили на текущем шаге'],
                          'Рабочий процесс' : row['Рабочий процесс'],
                          'Жизненный цикл' : row['Жизненный цикл']
                          })
    result_df_out = pd.DataFrame(result_df)
    with pd.ExcelWriter('out_result_df.xlsx',engine = 'xlsxwriter') as writer:
        result_df_out.to_excel(writer, index = False)


        workbook = writer.book
        worksheet = writer.sheets['Sheet1']

        worksheet.set_column('B:B', 20)
    # result_df_out = pd.DataFrame(result_df)
    # result_df_out.to_excel('out_result_df.xlsx', index = False, engine = 'openpyxl')

    return overdue_counts, overdue_emails, overdue_coordination_ids, coordination_details, debug_info, ambiguous_matches, matching_log


def save_results(overdue_counts, overdue_emails, overdue_coordination_ids, coordination_details, debug_info, ambiguous_matches, matching_log):
    """Save results to files with ambiguous matches report"""
    try:
        # Save company statistics
        with open('results.txt', 'w', encoding='utf-8') as f:
            f.write(f"Количество просроченных согласований: {len(overdue_coordination_ids)}\n\n")
            for company, count in sorted(overdue_counts.items(), key=lambda x: x[1], reverse=True):
                f.write(f"Количество неотработанных согласований {company}: {count}\n")
        print("Saved results.txt")

        # Save emails for reminders
        with open('overdue_emails.txt', 'w', encoding='utf-8') as f:
            f.write("\n".join(sorted(set(overdue_emails))))
        print("Saved overdue_emails.txt")

        # Save overdue coordination IDs (convert to strings first)
        with open('overdue_coordination_ids.txt', 'w', encoding='utf-8') as f:
            f.write("\n".join(sorted(set(str(id) for id in overdue_coordination_ids))))
        print("Saved overdue_coordination_ids.txt")

        # Save coordination details
        with open('coordination_details.txt', 'w', encoding='utf-8') as f:
            f.write("Coordination ID\tCompany\tStart Date\tDeadline\tWorking Days\tNot Checked Count\tExplanation\tEmails\n")
            for detail in coordination_details:
                f.write(f"{detail['id']}\t{detail['company']}\t{detail['start_date']}\t{detail['deadline']}\t")
                f.write(f"{detail['working_days']}\t{detail['not_checked_count']}\t{detail['explanation']}\t{', '.join(detail['emails'])}\n")
        print("Saved coordination_details.txt")

        # Save coordination details to Excel file
        coordination_df = pd.DataFrame(coordination_details)
        if not coordination_df.empty:
            # Reorder columns for better readability
            coordination_df = coordination_df[['id', 'company', 'start_date', 'deadline', 'working_days',
                                             'not_checked_count', 'explanation', 'emails']]
            coordination_df.to_excel('coordination_details.xlsx', index=False, engine='openpyxl')
            print("Saved coordination_details.xlsx")



        # Save debug info
        with open('debug_info.txt', 'w', encoding='utf-8') as f:
            f.write("Coordination ID\tStatus\tStart Date\tDeadline\tWorking Days\tExplanation\tCompanies\n\n")
            for info in debug_info:
                f.write(f"{info['id']}\t{info['status']}\t")
                if 'start_date' in info:
                    f.write(f"{info['start_date']}\t{info['deadline']}\t")
                else:
                    f.write("N/A\tN/A\t")
                f.write(f"{info['working_days']}\t{info['explanation']}\t")
                f.write(f"{','.join(info.get('companies', [])) if 'companies' in info else 'N/A'}\n")
        print("Saved debug_info.txt")

        # Save ambiguous matches for manual review
        if ambiguous_matches:
            with open('ambiguous_matches.txt', 'w', encoding='utf-8') as f:
                f.write("Coordination ID\tApprover Name\tPossible Candidates\n")
                for item in ambiguous_matches:
                    candidates = " | ".join([
                        f"{c['name']} <{c['email']}> ({c['company']})"
                        for c in item['candidates']
                    ])
                    f.write(f"{item['coordination_id']}\t{item['approver_name']}\t{candidates}\n")
            print("Saved ambiguous_matches.txt")

        # Save matching log
        with open('matching_log.txt', 'w', encoding='utf-8') as f:
            f.write("\n".join(matching_log))
        print("Saved matching_log.txt")

        print("\n✓ Results saved:")
        print("- results.txt (company statistics)")
        print("- overdue_emails.txt (reminder emails)")
        print("- overdue_coordination_ids.txt (overdue coordination IDs)")
        print("- coordination_details.txt (detailed coordination info)")
        print("- coordination_details.xlsx (Excel file with coordination details)")
        print("- debug_info.txt (processing debug information)")
        print("- matching_log.txt (detailed matching process)")
        if ambiguous_matches:
            print("- ambiguous_matches.txt (matches needing manual review)")
    except Exception as e:
        print(f"⚠️ Error saving results: {e}")
        traceback.print_exc()


def read_file(file_path):
    """Read file (CSV or Excel) and return DataFrame"""
    try:
        if file_path.lower().endswith(('.csv', '.txt')):
            # Try different encodings for CSV
            try:
                return pd.read_csv(file_path, sep=';', encoding='utf-8')
            except UnicodeDecodeError:
                return pd.read_csv(file_path, sep=';', encoding='windows-1251')
        else:  # Excel
            return pd.read_excel(file_path, engine='openpyxl')
    except Exception as e:
        print(f"⚠️ Error reading file: {e}")
        traceback.print_exc()
        raise


def data_loading_mode():
    """Mode 1: Load and process employee data"""
    print("=== Data Loading Mode ===")
    print("This mode processes employee data and updates the database.")

    # Load employee database
    db = load_employee_db()

    # Get file path from user input
    print("\nUpload company and employee data file (CSV or Excel):")
    file_path = upload_file("company and employee data")
    if not file_path:
        return

    # Read file
    df = read_file(file_path)

    # Convert DataFrame to text for parse_company_person_data
    file_content = '\n'.join(df.astype(str).values.flatten().tolist())

    # Process data using enhanced matching
    db, company_person_map, processing_log = parse_company_person_data(file_content, db)

    # Save processing log
    with open('data_loading_log.txt', 'w', encoding='utf-8') as f:
        f.write("\n".join(processing_log))
    print("Saved data_loading_log.txt")

    print("\nData loading completed successfully!")
    print(f"Database now contains {len(db['employees'])} employees and {len(db['companies'])} companies")


def data_matching_mode():
    """Mode 2: Match coordination data with employee database"""
    print("=== Data Matching Mode ===")
    print("This mode processes coordination data using the existing database.")

    # Load employee database
    db = load_employee_db()

    if not db['employees']:
        print("No employee data found in database. Please run Data Loading mode first.")
        return

    # Convert database to company_person_map format
    company_person_map = defaultdict(list)
    for employee in db['employees']:
        company = employee['company']
        company_person_map[company].append({
            'name': employee['name'],
            'email': employee['email'],
            'normalized_name': employee['normalized_name'],
            'surname': employee['surname'],
            'given_names': employee['given_names'],
            'team_id': employee.get('team_id', ''),
            'team_emails': employee.get('team_emails', [])
        })

    # Get file path from user input
    print("\nUpload coordination data file (CSV or Excel):")
    file_path = upload_file("coordination data")
    if not file_path:
        return

    # Read file
    df = read_file(file_path)

    # Process coordinations
    overdue_counts, overdue_emails, overdue_coordination_ids, coordination_details, debug_info, ambiguous_matches, matching_log = process_coordinations(
        df, company_person_map
    )

    # Print coordination details to console for debugging
    print("\n" + "=" * 50)
    print("Coordination Processing Details (Enhanced Matching):")
    print("=" * 50)
    for detail in coordination_details:
        print(f"ID: {detail['id']}")
        print(f"  Company: {detail['company']}")
        print(f"  Start Date: {detail['start_date']}")
        print(f"  Deadline: {detail['deadline']}")
        print(f"  Working Days: {detail['working_days']}")
        print(f"  Explanation: {detail['explanation']}")
        print(f"  Not Checked Count: {detail['not_checked_count']}")
        print(f"  Emails: {', '.join(detail['emails'])}")
        print("-" * 50)

    save_results(overdue_counts, overdue_emails, overdue_coordination_ids, coordination_details, debug_info, ambiguous_matches, matching_log)

    print("\nFinal results:")
    print("=" * 50)
    print("Overdue coordination count by company:")
    for company, count in sorted(overdue_counts.items(), key=lambda x: x[1], reverse=True):
        print(f"- {company}: {count}")

    # Convert IDs to strings for display
    overdue_ids_str = ', '.join(str(id) for id in overdue_coordination_ids)
    print(f"\nOverdue coordination IDs: {overdue_ids_str}")

    if ambiguous_matches:
        print("\n⚠️ Some matches were ambiguous. Please review ambiguous_matches.txt")

    print("\nData matching completed successfully!")
    if(no_match_array):
      print(f"    !!! Some People were not found in data: {set(no_match_array)}")


def main():
    print("=== Coordination Processing System ===")

    # Check for required packages
    try:
        import pandas as pd
    except ImportError:
        print("Please install pandas first: pip install pandas")
        return

    try:
        import openpyxl
    except ImportError:
        print("Please install openpyxl for Excel support: pip install openpyxl")
        return

    try:
        from rapidfuzz import fuzz
    except ImportError:
        print("Please install rapidfuzz for fuzzy matching: pip install rapidfuzz")
        return

    while True:
        print("\nSelect mode:")
        print("1. Data Loading - Process employee data and update database")
        print("2. Data Matching - Process coordination data using existing database")
        print("3. Upload employee database JSON file")
        print("4. Exit")

        choice = input("Enter your choice (1-4): ").strip()

        if choice == '1':
            data_loading_mode()
        elif choice == '2':
            data_matching_mode()
        elif choice == '3':
            # Upload employee database
            uploaded_file = upload_file("employee database JSON")
            if uploaded_file and uploaded_file.endswith('.json'):
                # Copy the uploaded file to the expected location
                import shutil
                shutil.copy(uploaded_file, EMPLOYEE_DB_FILE)
                print(f"✓ Employee database uploaded and saved as {EMPLOYEE_DB_FILE}")
            else:
                print("Please upload a valid JSON file")
        elif choice == '4':
            print("Exiting program. Goodbye!")
            break
        else:
            print("Invalid choice. Please try again.")


if __name__ == "__main__":
    main()

Fatal Python error: init_import_site: Failed to import the site module
Python runtime state: initialized
Traceback (most recent call last):
  File "<frozen importlib._bootstrap>", line 1360, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1331, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 935, in _load_unlocked
  File "<frozen importlib._bootstrap>", line 1176, in exec_module
  File "<frozen site>", line 652, in <module>
  File "<frozen site>", line 639, in main
  File "<frozen site>", line 421, in addsitepackages
  File "<frozen site>", line 253, in addsitedir
  File "<frozen site>", line 212, in addpackage
object address  : 0x7fc0f9802260
object refcount : 1
object type     : 0xa2a4e0
object type name: KeyboardInterrupt
object repr     : KeyboardInterrupt()
lost sys.stderr
  File "<string>", line 1, in <module>
=== Coordination Processing System ===

Select mode:
1. Data Loading - Process employee data and update database
2. Data Matching - P

Saving reviews_export-31-10-2025.xlsx to reviews_export-31-10-2025 (1).xlsx
✓ Uploaded: reviews_export-31-10-2025 (1).xlsx

Processing company and employee data...
✓ Saved employee database with 21 employees and 8 companies
Saved data_loading_log.txt

Data loading completed successfully!
Database now contains 21 employees and 8 companies

Select mode:
1. Data Loading - Process employee data and update database
2. Data Matching - Process coordination data using existing database
3. Upload employee database JSON file
4. Exit
Enter your choice (1-4): 2
=== Data Matching Mode ===
This mode processes coordination data using the existing database.
✓ Loaded employee database with 21 employees and 8 companies

Upload coordination data file (CSV or Excel):
Please upload coordination data file:


Saving reviews_export-31-10-2025.xlsx to reviews_export-31-10-2025 (2).xlsx
✓ Uploaded: reviews_export-31-10-2025 (2).xlsx



Type date in format dd.mm.YYYY 
or pull Enter if today is ok: 

Processing coordinations (as of 2025-11-07)... 2025-11-07 17:54:54.108206
Using 'id' as coordination ID

Coordination Processing Details (Enhanced Matching):
ID: 282622
  Company: bim-info
  Start Date: 2025-10-30
  Deadline: 2025-11-04
  Working Days: 3
  Explanation: Stage 2: not configured, using default 3 days
  Not Checked Count: 1
  Emails: akolosova@bim-info.ru
--------------------------------------------------
ID: 280308
  Company: su-19, fodd, c-and-u
  Start Date: 2025-10-29
  Deadline: 2025-11-05
  Working Days: 5
  Explanation: Stage 3: not configured, using default 5 days
  Not Checked Count: 3
  Emails: dkornakov@fodd.ru, mzemtsov@c-and-u.co, kiseleva@su-19.com
--------------------------------------------------
ID: 280301
  Company: su-19, fodd, c-and-u
  Start Date: 2025-10-29
  Deadl

In [ ]:
from google.colab import drive
drive.mount('/content/drive')